In [1]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import dfca as DFCA
import ifca as IFCA

import numpy as np

from util import *

Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [2]:
rc = ipp.Cluster(n=2).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [3]:
seeds = [11,12,13,14,15]
learning_rates = [0.1]

dfca_gi = []
ifca = []

In [4]:
%%px --targets all

import sys
import os
sys.path.append(os.getcwd())

seeds = [11,12,13,14,15]
learning_rates = [0.1]

dfca_gi = []
ifca = []

Running the dfca-gi experiment

In [5]:
%%px --target 0 --noblock
import dfca as DFCA
import json
import time
import torch
import numpy as np


with open("config_dfca_gi.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-GI, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [6]:
%%px --target 1 --noblock
import ifca as IFCA
import json
import time
import torch
import numpy as np

with open("config_ifca.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------IFCA, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = IFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    ifca.append(res_mean)



<AsyncResult(%px): pending>

## Results

DFCA-GI Experiment Outputs

In [7]:
rc.wait_interactive()
_.display_outputs()

dfca_gi = rc[0].pull('dfca_gi', block=True)
ifca = rc[1].pull('ifca', block=True)

unknown:   0%|          | 0/2 [00:00<?, ?tasks/s]

Output for 1:
2025-05-27 17:26:54.446 [IPEngine] Loading connection info from $IPP_CONNECTION_INFO
2025-05-27 17:26:54.446 [IPEngine] WARNING | Not using CurveZMQ security
2025-05-27 17:26:54.448 [IPEngine] Registering with controller at tcp://127.0.0.1:37639
2025-05-27 17:26:54.450 [IPEngine.1] Shell_addrs: ['tcp://127.0.0.1:39429', 'tcp://127.0.0.1:54803', 'tcp://127.0.0.1:40923']
2025-05-27 17:26:54.450 [IPEngine.1] Connecting shell to tcp://127.0.0.1:39429
2025-05-27 17:26:54.450 [IPEngine.1] Connecting shell to tcp://127.0.0.1:54803
2025-05-27 17:26:54.450 [IPEngine.1] Connecting shell to tcp://127.0.0.1:40923
2025-05-27 17:26:54.450 [IPEngine.1] Starting nanny
2025-05-27 17:26:54.763 [KernelNanny.1] Starting kernel nanny for engine 1, pid=305725, nanny pid=305740
2025-05-27 17:26:54.764 [KernelNanny.1] Nanny watching parent pid 305725.
2025-05-27 17:26:54.819 [IPEngine.1] Loading IPython extension: storemagic
2025-05-27 17:26:54.821 [IPEngine.1] WARNING | debugpy_stream undefined

---------------------------------IFCA, lr: 0.1, seed: 11---------------------------------
config: {'m': 2400, 'm_test': 400, 'p': 4, 'n': 100, 'uneven': True, 'h1': 200, 'num_epochs': 300, 'batch_size': 100, 'tau': 10, 'lr': 0.1, 'data_seed': 11, 'train_seed': 11, 'config_override': '', 'project_dir': 'output_ifca'}
Using device: cuda
Epoch -1 tr: l 2.291 a 0.117 clct[851, 530, 732, 287] cl_acc 0.410  3.782sec
Epoch -1 tst: l 2.292 a 0.115 clct[150, 78, 135, 37] cl_acc 0.403  0.874sec
Epoch 0 tr: l 2.191 a 0.283 clct[549, 586, 668, 597] cl_acc 0.819  lr 0.100000 18.222sec(train) 6.510sec(infer)
Epoch 0 tst: l 2.191 a 0.285 clct[91, 100, 116, 93] cl_acc 0.812  1.200sec
result written at output_ifca/results.pickle
checkpoint written at output_ifca/checkpoint.pt
Epoch 1 tr: l 1.944 a 0.577 clct[599, 602, 598, 601] cl_acc 0.991  lr 0.100000 14.357sec(train) 5.751sec(infer)
Epoch 1 tst: l 1.940 a 0.577 clct[100, 101, 101, 98] cl_acc 0.995  0.912sec
Epoch 2 tr: l 1.601 a 0.729 clct[600, 600,

engine set stopped 1748359614: {'engines': {'1': {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 305725, 'identifier': '1'}, '0': {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 305723, 'identifier': '0'}}, 'exit_code': <Negsignal.SIGKILL: -9>}


NoEnginesRegistered: This operation requires engines. Try client.wait_for_engines(n) to wait for engines to register.

IFCA Experiment Outputs

In [8]:
__.display_outputs()

---------------------------------DFCA-GI, lr: 0.1, seed: 11---------------------------------
config: {'m': 2400, 'm_test': 400, 'p': 4, 'n': 100, 'uneven': True, 'local_model_init': False, 'h1': 200, 'num_epochs': 300, 'batch_size': 100, 'tau': 10, 'lr': 0.1, 'data_seed': 11, 'train_seed': 11, 'config_override': '', 'project_dir': 'output_dfca_gi'}
Using device: cuda
Epoch -1 tr: l 2.291 a 0.117 clct[851, 530, 732, 287] cl_acc 0.410  4.648sec
Epoch -1 tst: l 2.292 a 0.141 clct[150, 78, 135, 37] cl_acc 0.403  3.294sec
Epoch 0 tr: l 2.146 a 0.329 clct[593, 603, 671, 533] cl_acc 0.706  lr 0.100000 34.153sec(train) 5.534sec(infer)
Epoch 0 tst: l 2.190 a 0.323 clct[97, 90, 130, 83] cl_acc 0.700  4.275sec
result written at output_dfca_gi/results.pickle
checkpoint written at output_dfca_gi/checkpoint.pt
Epoch 1 tr: l 1.914 a 0.568 clct[591, 609, 603, 597] cl_acc 0.933  lr 0.100000 35.646sec(train) 5.741sec(infer)
Epoch 1 tst: l 1.976 a 0.534 clct[95, 96, 115, 94] cl_acc 0.955  4.316sec
Epoch 

IFCA Experiment Outputs

## Plots

In [9]:

r_ifca = {}
r_dfca_gi = {}

r_ifca['test_loss'] = []
r_ifca['test_acc'] = []
r_ifca['cl_acc'] = []
r_ifca['train_loss'] = []
for i in range(len(learning_rates)):
    r_ifca['test_loss'].append(ifca[i][0])
    r_ifca['test_acc'].append(ifca[i][1])
    r_ifca['cl_acc'].append(ifca[i][2])
    r_ifca['train_loss'].append(ifca[i][3])

r_dfca_gi['test_loss'] = []
r_dfca_gi['test_acc'] = []
r_dfca_gi['cl_acc'] = []
r_dfca_gi['train_loss'] = []
for i in range(len(learning_rates)):
    r_dfca_gi['test_loss'].append(dfca_gi[i][0])
    r_dfca_gi['test_acc'].append(dfca_gi[i][1])
    r_dfca_gi['cl_acc'].append(dfca_gi[i][2])
    r_dfca_gi['train_loss'].append(dfca_gi[i][3])



os.makedirs('graphs', exist_ok=True)

for l in range(len(learning_rates)):

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['train_loss'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['train_loss'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(f'Train Loss per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_train_loss.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['test_loss'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['test_loss'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(f'Test Loss per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_test_loss.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['test_acc'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['test_acc'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('test accuracy')
    plt.title(f'Test Accuracy per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_test_acc.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['cl_acc'][l][:31], label='ifca', color='blue')
    plt.plot(r_dfca_gi['cl_acc'][l][:31], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('cluster acc')
    plt.title(f'Cluster Accuracy per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_cluster_acc.png'))

IndexError: list index out of range